## **Import the TensorFlow library**

This code contains a small script that is used for generating a folder full of images. The idea is to show these images to the OpenMV camera when the training with the OL methos is required. Since the idea is to apply a supervised training is required to have a ground truth forthe computation of the error that the model is doing. This script fills a directory with images of digits and creates a txt file in which all the labels are saved. 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image

import numpy as np
import pandas as pd
import seaborn as sns
import random 
import csv 
import sys
import os
import re
from random import seed

import time
#import os, os.path
import cv2
import glob
from keras import applications
#from keras.applications import vgg19
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score

from numpy.ma.core import size

import torchvision.models as models
from torchvision import transforms
ROOT_PATH = os.path.abspath('')
sys.path.insert(0, ROOT_PATH + '/lib')

# VS Code

# IMPORT AND LOAD MODELS

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans


import random 
from random import seed

import time


import numpy as np
import matplotlib.pyplot as plt


# Load features and labels
features_saved = np.loadtxt('Models/Original_model/ll_features_10.txt')
labels_features_saved = np.loadtxt('Models/Original_model/ll_labels_features_10.txt').astype(int)

# Load model - CFR
model = keras.models.load_model('Models/Original_model/mnist_cnn.h5') # Frozen model 

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')



# Test import
# from lib import simulation_lib
# from lib.simulation_lib import

from lib import Kmeans_lib
from lib.Kmeans_lib import *
from lib.EvalMetrics import *


2022-10-18 22:01:46.529926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 22:01:52.434175: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# LOAD DATASETS

In [2]:
(data_train, label_train),(data_test, label_test) = mnist.load_data() # Load data
print('The original dataset shapes from MNIST are')
print(f'    Train dataset shape: {data_train.shape}')
print(f'    Test dataset shape:  {data_test.shape}')

The original dataset shapes from MNIST are
    Train dataset shape: (60000, 28, 28)
    Test dataset shape:  (10000, 28, 28)


In [3]:
train_samples = label_train.shape[0]
test_samples  = label_test.shape[0]
img_rows, img_cols = 28, 28

trainLow_samples = np.sum(np.where(label_train < 6, 1, 0))
testLow_samples  = np.sum(np.where(label_test  < 6, 1, 0))

# separate in containers data that is lower and higer than 6
# TRAIN - LOW
data_low_train   = np.zeros([trainLow_samples,28,28])
label_low_train  = np.zeros(trainLow_samples)
#       - HIGH
data_high_train  = np.zeros([train_samples-trainLow_samples,28,28])
label_high_train = np.zeros(train_samples-trainLow_samples)

# TEST - LOW
data_low_test   = np.zeros([testLow_samples,28,28])
label_low_test  = np.zeros(testLow_samples)

#      - HIGH
data_high_test  = np.zeros([test_samples-testLow_samples,28,28])
label_high_test = np.zeros(test_samples-testLow_samples)

j,k = 0,0
for i in range(0,train_samples):  
    if(label_train[i]<6):
        data_low_train[j,:,:] = data_train[i,:,:]
        label_low_train[j]    = label_train[i]
        j+=1
    else:
        data_high_train[k,:,:] = data_train[i,:,:]
        label_high_train[k]    = label_train[i]
        k+=1


j,k = 0,0
for i in range(0,test_samples):
    if(label_test[i]>5):
        data_high_test[k,:,:] = data_test[i,:,:]
        label_high_test[k]    = label_test[i]
        k+=1  
    else:
        data_low_test[j,:,:] = data_test[i,:,:]
        label_low_test[j]    = label_test[i]
        j+=1

# Reshape arrays
data_low_train  = data_low_train.reshape(data_low_train.shape[0], img_rows, img_cols, 1)
data_high_train = data_high_train.reshape(data_high_train.shape[0], img_rows, img_cols, 1)
data_low_test   = data_low_test.reshape(data_low_test.shape[0], img_rows, img_cols, 1)
data_high_test  = data_high_test.reshape(data_high_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
    
print('After the separation of the dataset in groups higer and lower/equal than 6 the datasets are:')
print(f'     Train dataset lower than 6 has shape:  {data_low_train.shape}')
print(f'     Train dataset higher than 6 has shape: {data_high_train.shape}')
print()
print(f'     Test dataset lower than 6 has shape:  {data_low_test.shape}')
print(f'     Test dataset higher than 6 has shape: {data_high_test.shape}')

# Normalize the colors from 0-255 to 0-1
data_low_train  = data_low_train.astype(np.float32) / 255.0
data_high_train = data_high_train.astype(np.float32) / 255.0
data_low_test   = data_low_test.astype(np.float32) / 255.0
data_high_test  = data_high_test.astype(np.float32) / 255.0

After the separation of the dataset in groups higer and lower/equal than 6 the datasets are:
     Train dataset lower than 6 has shape:  (36017, 28, 28, 1)
     Train dataset higher than 6 has shape: (23983, 28, 28, 1)

     Test dataset lower than 6 has shape:  (6031, 28, 28, 1)
     Test dataset higher than 6 has shape: (3969, 28, 28, 1)


In [4]:
n_samples = 400
digits_train = np.zeros((n_samples,28,28))
digits_test = np.zeros((n_samples,28,28))
label_digits_train = np.zeros(n_samples)
label_digits_test = np.zeros(n_samples)

for i in range(0, n_samples):
  n = random.randint(0,len(data_train)-1)
  digits_train[i,:,:] = data_train[n,:,:]
  label_digits_train[i] = label_train[n]
  m = random.randint(0,len(data_test)-1)
  digits_test[i,:,:] = np.copy(data_test[m,:,:])
  label_digits_test[i] = label_test[m]


digits_train  = digits_train.reshape(digits_train.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
digits_test = digits_test.reshape(digits_test.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
input_shape = (img_rows, img_cols, 1)

print(f'The dataset on which the model will be TRAINED has shape {digits_train.shape}') # da cambiare con il nuovo set
print(f'The dataset on which the model will be TESTED has shape  {digits_test.shape}') # da cambiare con il nuovo set

The dataset on which the model will be TRAINED has shape (400, 28, 28, 1)
The dataset on which the model will be TESTED has shape  (400, 28, 28, 1)


# FUNCTIONS

# TRAIN THE MODEL ON THE DIGITS 6-9 (OL METHOD)

In [22]:
def run_frozen():
    pass

def update_active_layer(model, features_images, pseudo_labels):

    learn_rate = model.l_rate

    CheckLabelKnown(model, pseudo_labels[i])
    
    y_true_soft = DigitToSoftmax(pseudo_labels[i], model.label)
               
    # Prediction
    y_pred = model.predict(features_images[i,:])
        
    # Backpropagation
    cost = y_pred-y_true_soft
        
    for j in range(0,model.W.shape[0]):

         # Update weights
        dW = np.multiply(cost, features_images[i,j]*learn_rate)
        model.W[j,:] = model.W[j,:]-dW

    # Update biases
    db      = np.multiply(cost, learn_rate)
    model.b = model.b-db
           
    # the next part is only to plot the confusion matrix
    # if the train data is finished still train the model but do not save the results
    if(i>=train_samples):

        y_true_soft = DigitToSoftmax(y_tot[i], model.label)
                   
        # Find the max iter for both true label and prediction
        if(np.amax(y_true_soft) != 0):
            max_i_true = np.argmax(y_true_soft)

        if(np.amax(y_pred) != 0):
            max_i_pred = np.argmax(y_pred)

def trainOneEpoch_OL(model, images, labels, features_saved, labels_features_saved, batch_size):
    
    n_cluster = 10

    # cntr_clus = 0
    # cntr = 1 #  A COSA SERVE?
    # tot_samples = images.shape[0]

    # Features extraction
    # features = model.ML_frozen.predict(images.reshape((1,28,28,1)), verbose = False)

    # Split dataset in batches
    n_batch = int(np.ceil(images.shape[0] / batch_size))
    images_batch = np.array_split(images, n_batch)
    labels_batch = np.array_split(labels, n_batch)

    err_tot = 0
    pseudo_labels = np.empty
    for i in range(0, n_batch):
        print("Starting batch:" + str(i) + "/" + str(n_batch))
        # Features extraction
        start1 = time.time()
        features = model.ML_frozen.predict(images_batch[i].reshape((batch_size,28,28,1)), verbose = False) # VERIFICO
        end1 = time.time()

        # Kmean clustering
        start2 = time.time()
        pseudo_label, err = k_mean_clustering(features, features_saved, labels, labels_features_saved, n_cluster, batch_size)
        end2 = time.time()
        pseudo_labels = np.append(pseudo_labels, pseudo_label.astype(int))
        err_tot += err

        print("Features extraction took {:f} seconds".format(end1 - start1) + " and Kmean clustering took {:f} seconds".format(end2 - start2) + ", with {} accuracy".format(err/features.shape[0]))

    # 


    

    # Check pseudo-labels and errors in clustering
    
    #print("Pseudo_labels vector: ")
    #print(pseudo_labels)
    #print("\n")

    print("The error in clustering is: ")
    print(int(err_tot/images.shape[0]*100), '%')
    print("errors:", err_tot)
    print("tot samples", images.shape[0])

    # ONLINE-LEARNING

    print('**********************************\n Performing training with OL\n')

    for i in range(0, tot_samples):
        update_active_layer()


In [23]:
Model_OL = Custom_Layer(model)
Model_OL.title      = 'OL'
Model_OL.filename   = 'OL'
Model_OL.l_rate     = 0.01
Model_OL.batch_size = 8

batch_size = 40 # 20

import importlib
importlib.reload(Kmeans_lib)

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

# Merge train and test datasets
digits = np.concatenate((digits_train, digits_test))   
labels = np.concatenate((label_train, label_test))  

trainOneEpoch_OL(Model_OL, digits, labels, features_saved, labels_features_saved, batch_size)

Starting batch:0/20
Features extraction took 0.138198 seconds and Kmean clustering took 1.782408 seconds, with 0.975 accuracy
Starting batch:1/20
Features extraction took 0.065929 seconds and Kmean clustering took 0.564417 seconds, with 0.9 accuracy
Starting batch:2/20
Features extraction took 0.090907 seconds and Kmean clustering took 1.170998 seconds, with 0.9 accuracy
Starting batch:3/20
Features extraction took 0.065238 seconds and Kmean clustering took 0.567339 seconds, with 0.825 accuracy
Starting batch:4/20
Features extraction took 0.068097 seconds and Kmean clustering took 3.544659 seconds, with 0.825 accuracy
Starting batch:5/20
Features extraction took 0.063365 seconds and Kmean clustering took 1.759347 seconds, with 0.9 accuracy
Starting batch:6/20
Features extraction took 0.063979 seconds and Kmean clustering took 1.985459 seconds, with 0.975 accuracy
Starting batch:7/20


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 180, in where
KeyboardInterrupt: 


Features extraction took 0.070932 seconds and Kmean clustering took 12.200978 seconds, with 0.85 accuracy
Starting batch:8/20
Features extraction took 0.064151 seconds and Kmean clustering took 0.617882 seconds, with 0.875 accuracy
Starting batch:9/20
Features extraction took 0.065012 seconds and Kmean clustering took 0.753349 seconds, with 0.95 accuracy
Starting batch:10/20
Features extraction took 0.064463 seconds and Kmean clustering took 1.447536 seconds, with 0.9 accuracy
Starting batch:11/20
Features extraction took 0.064425 seconds and Kmean clustering took 33.746172 seconds, with 0.925 accuracy
Starting batch:12/20
Features extraction took 0.066111 seconds and Kmean clustering took 2.205365 seconds, with 0.925 accuracy
Starting batch:13/20
Features extraction took 0.077313 seconds and Kmean clustering took 0.797837 seconds, with 0.925 accuracy
Starting batch:14/20
Features extraction took 0.073576 seconds and Kmean clustering took 0.811452 seconds, with 0.85 accuracy
Starting b

NameError: name 'tot_samples' is not defined

In [ ]:
feat = features_saved[1,:]

##(features_saved.all)
#max(features_saved)

feat.dtype

np.amin(feat)
np.amax(feat)

feat.shape
feat

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.042, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.245, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.007, 0.715, 0.   , 0.   , 0.   , 0.524, 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.777, 0.053, 0.9  , 0.   , 0.   ,
       0.   , 0.972, 0.   , 0.   , 0.   , 1.784, 0.   , 0.   , 1.024,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.714, 2.935, 0.   ,
       0.   , 0.988, 2.111, 0.   , 0.   , 1.147, 0.   , 0.   , 0.463,
       1.97 , 0.   , 2.253, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.163, 0.698, 2.776, 0.   , 0.   , 0.   , 0.   , 2.963, 0.   ,
       1.29 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 2.057,
       0.   , 0.   , 0.   , 0.462, 0.017, 0.   , 0.   , 0.   , 0.   ,
       0.278, 0.   , 0.   , 0.   , 0.   , 0.   , 0.686, 1.092, 0.   ,
       2.389, 0.   , 0.   , 1.416, 0.   , 1.505, 0.   , 1.37 , 0.   ,
       0.   , 0.   ,

In [ ]:
X = digits_train[1,:]
YY = Model_OL.ML_frozen.predict(X.reshape((1,28,28,1)), verbose = False)

# YY2 = np.array(YY as features_saved)

np.amin(YY)
np.amax(YY)

YY.shape

np.set_printoptions(suppress=True)

YY

array([[0.        , 0.96312505, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.3460795 , 1.0170912 ,
        0.8546268 , 0.        , 0.        , 0.        , 0.79275155,
        0.57086986, 0.        , 0.93478656, 3.351025  , 0.        ,
        0.        , 1.7612993 , 0.        , 1.2195386 , 0.        ,
        0.        , 0.        , 0.5815387 , 0.64130604, 2.1060772 ,
        0.        , 0.        , 0.        , 2.6718721 , 0.        ,
        0.        , 4.112043  , 0.        , 0.        , 2.0145454 ,
        1.4062762 , 0.        , 4.5407233 , 0.        , 0.        ,
        0.72269887, 0.        , 0.        , 0.        , 0.        ,
        0.60950524, 3.2392838 , 0.        , 0.        , 1.0193931 ,
        2.779964  , 3.569485  , 0.        , 0.2462171 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.8287938 ,
        1.0599822 , 1.351576  , 0.        , 0.21574514, 0.        ,
        0.        , 0.        , 0.        , 0.  

In [ ]:



#from simulation_lib import *

#plot_barChart(Model_OL)
#plot_confMatrix(Model_OL)
#plot_table(Model_OL)

# from lib import simulation_lib as sim_lib
# sim_lib.plot_barChart(Model_OL)
# sim_lib.plot_confMatrix(Model_OL)
# sim_lib.plot_table(Model_OL)

In [ ]:



############

# Dummy  main for debug
if False:
  n = range(0, 10)
  images = data_train[n,:,:]
  labels_features_run = label_train[n]

  features_run = []
  for img in images:
    features_run.append(Custom_Layer(model).ML_frozen.predict(img.reshape((1,28,28,1)))/1000)

  features_run = np.array(features_run).squeeze(axis=1)

  n_cluster = 5 # ARGOMENTO NON USATO
  # AGGIORNARE PASSANDO LA LISTA DI LABELS DA CLUSTERIZZARE
  batch_size = 10

  kmean = k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size)